In [2]:
pip install flair

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.Collecting segtok>=1.5.7


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 4.11.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.



  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14759 sha256=1bd0b6ddc4cf322dc8717efaca17ef9e7005d976cae04121b3f9bd7e845b6408
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\fb\c3\0e\c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116678 sha256=e244ec82d5ec78eaa9807edead7fc456403c3c2570434f4ce499b3f369088eb9
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\26\70\6a\1c79e59951a41b4045497da187b2724f5659ca64033cf4548e
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10175 sha256=e10a8b7db41649bfa146ccf14d8a1f8d254bc74d5ed7105b051c29ebf3fb1f48
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\3a\0d\38\01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
  Created wheel for sqlitedict: filename=sqlitedict-2.0.0-py3-none-any.whl size=15718 sha256=11ddbe8fb3a6d8309dd553ac2b903ffb830f2987ba251e0946b9bc90a3d404d7
  Stor

In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

## Q, ME, MP, Qlr Tagging

In [4]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train and test files reside
data_folder = 'C:/Users/DELL/Desktop/MeasEval/NER_sent'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              dev_file='dev.txt',
                              test_file='test.txt')

2022-08-11 14:30:18,110 Reading data from C:\Users\DELL\Desktop\MeasEval\NER_sent
2022-08-11 14:30:18,112 Train: C:\Users\DELL\Desktop\MeasEval\NER_sent\train.txt
2022-08-11 14:30:18,114 Dev: C:\Users\DELL\Desktop\MeasEval\NER_sent\dev.txt
2022-08-11 14:30:18,116 Test: C:\Users\DELL\Desktop\MeasEval\NER_sent\test.txt


In [5]:
len(corpus.train)

1943

In [6]:
print(corpus.train[0].to_tagged_string('ner'))

Sentence: "Data were drawn from the Whitehall II study with baseline examination in 1991 ; follow - up screenings in 1997 , 2003 , and 2008 ; and additional disease ascertainment from hospital data and registry linkage on 5318 participants ( mean age 54 . 8 years , 31 % women ) without depressive symptoms at baseline ." → ["Whitehall II study"/MeasuredEntity, "baseline examination"/MeasuredEntity, "1991"/Quantity, "follow - up screenings"/MeasuredEntity, "1997 , 2003 , and 2008"/Quantity, "5318 participants"/MeasuredEntity, "mean age"/MeasuredProperty, "54 . 8 years"/Quantity, "31 %"/Quantity, "women"/MeasuredProperty]


In [7]:
len(corpus.test)

561

In [8]:
len(corpus.dev)

281

In [9]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [10]:
# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# # 4. initialize fine-tuneable transformer embeddings WITH document context
# embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
#                                        layers="-1",
#                                        subtoken_pooling="first",
#                                        fine_tune=True,
#                                        use_context=True,
#                                        )

# # 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
# tagger = SequenceTagger(hidden_size=256,
#                         embeddings=embeddings,
#                         tag_dictionary=label_dict,
#                         tag_type='ner',
#                         use_crf=False,
#                         use_rnn=False,
#                         reproject_embeddings=False,
#                         )

# # 6. initialize trainer
# trainer = ModelTrainer(tagger, corpus)

# # 7. run fine-tuning
# trainer.fine_tune('resources/taggers/sota-ner-flert',
#                   learning_rate=5.0e-6,
#                   mini_batch_size=4,
#                   mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
#                   )

2022-08-11 14:30:30,664 Computing label dictionary. Progress:


1943it [00:00, 14931.19it/s]

2022-08-11 14:30:30,807 Dictionary created for label 'ner' with 5 values: Quantity (seen 1124 times), MeasuredEntity (seen 907 times), MeasuredProperty (seen 644 times), Qualifier (seen 90 times)
Dictionary with 5 tags: <unk>, Quantity, MeasuredEntity, MeasuredProperty, Qualifier


In [12]:
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='dslim/bert-base-NER',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       allow_long_sentences=True,
                                       use_context=True,
                                       layer_mean=False
                                       )

In [13]:
# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ner',
                        use_crf=True,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

2022-08-11 16:51:23,152 SequenceTagger predicts: Dictionary with 17 tags: O, S-Quantity, B-Quantity, E-Quantity, I-Quantity, S-MeasuredEntity, B-MeasuredEntity, E-MeasuredEntity, I-MeasuredEntity, S-MeasuredProperty, B-MeasuredProperty, E-MeasuredProperty, I-MeasuredProperty, S-Qualifier, B-Qualifier, E-Qualifier, I-Qualifier


In [14]:
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

In [ ]:
# 7. run fine-tuning
trainer.fine_tune('resources/taggers/ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4
                  )

2022-08-11 17:47:59,372 ----------------------------------------------------------------------------------------------------
2022-08-11 17:47:59,415 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): D

2022-08-11 17:47:59,426 ----------------------------------------------------------------------------------------------------
2022-08-11 17:47:59,438 Corpus: "Corpus: 1943 train + 281 dev + 561 test sentences"
2022-08-11 17:47:59,441 ----------------------------------------------------------------------------------------------------
2022-08-11 17:47:59,444 Parameters:
2022-08-11 17:47:59,446  - learning_rate: "0.000005"
2022-08-11 17:47:59,449  - mini_batch_size: "4"
2022-08-11 17:47:59,452  - patience: "3"
2022-08-11 17:47:59,455  - anneal_factor: "0.5"
2022-08-11 17:47:59,458  - max_epochs: "10"
2022-08-11 17:47:59,461  - shuffle: "True"
2022-08-11 17:47:59,464  - train_with_dev: "False"
2022-08-11 17:47:59,467  - batch_growth_annealing: "False"
2022-08-11 17:47:59,470 ----------------------------------------------------------------------------------------------------
2022-08-11 17:47:59,473 Model training base path: "resources\taggers\ner-flert"
2022-08-11 17:47:59,477 --------------

## Unit Tagging

In [2]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train and test files reside
data_folder = 'C:/Users/DELL/Desktop/MeasEval/NER_sent/data_unit'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              dev_file='dev.txt',
                              test_file='test.txt')

2022-08-14 20:45:50,284 Reading data from C:\Users\DELL\Desktop\MeasEval\NER_sent\data_unit
2022-08-14 20:45:50,289 Train: C:\Users\DELL\Desktop\MeasEval\NER_sent\data_unit\train.txt
2022-08-14 20:45:50,291 Dev: C:\Users\DELL\Desktop\MeasEval\NER_sent\data_unit\dev.txt
2022-08-14 20:45:50,293 Test: C:\Users\DELL\Desktop\MeasEval\NER_sent\data_unit\test.txt


In [3]:
# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-08-14 20:45:57,972 Computing label dictionary. Progress:


1943it [00:00, 11920.87it/s]

2022-08-14 20:45:58,201 Dictionary created for label 'ner' with 2 values: unit (seen 1051 times)
Dictionary with 2 tags: <unk>, unit


In [4]:
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='dslim/bert-base-NER',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       allow_long_sentences=True,
                                       use_context=True,
                                       layer_mean=False
                                       )

In [5]:
# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type='ner',
                        use_crf=True,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

2022-08-14 20:48:41,277 SequenceTagger predicts: Dictionary with 2 tags: <unk>, unit


In [6]:
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

In [ ]:
# 7. run fine-tuning
trainer.fine_tune('resources/taggers/ner-unit-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4
                  )